# nanochat

## 環境構築

In [ ]:
# RustBPEのビルドとインストール

%pip install maturin
import os

if not os.path.exists("nanochat"):
    !git clone https://github.com/karpathy/nanochat

try:
    # Google Colabの場合
    from google.colab import userdata

    # ターミナルでRustをインストール
    # curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y && . "$HOME/.cargo/env"

    # ターミナルでRustBPEのビルドとインストール
    # maturin build --release --manifest-path nanochat/rustbpe/Cargo.toml

    # ターミナルで生成されたwhlファイルをインストール
    # pip install nanochat/rustbpe/target/wheels/*.whl

    if not os.path.exists("nanochat/rustbpe/target"):
        raise FileNotFoundError("rustbpeのビルドとインストールをターミナルで実行してください。")

except ImportError:
    # ローカル環境の場合
    !maturin develop --release --manifest-path nanochat/rustbpe/Cargo.toml

import rustbpe

In [ ]:
# ログ設定

import logging as logging

if os.path.exists('debug.log'):
    os.remove('debug.log')

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = '🟦'
        case logging.INFO:
            level = '🟩'
        case logging.WARNING:
            level = '🟨'
        case logging.ERROR:
            level = '🟥'
        case logging.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler('debug.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)
logger.info("ログを初期化")

In [ ]:
# ベースディレクトリを設定

def get_base_dir():
    """
    ベースディレクトリのパスを取得する
    デフォルトは、~/.cache/nanochat
    NANOCHAT_BASE_DIR環境変数で上書き可能

    Returns:
        str: ベースディレクトリのパス
    """
    if os.environ.get("NANOCHAT_BASE_DIR"):
        nanochat_dir = os.environ.get("NANOCHAT_BASE_DIR")
    else:
        home_dir = os.path.expanduser("~")
        cache_dir = os.path.join(home_dir, ".cache")
        nanochat_dir = os.path.join(cache_dir, "nanochat")
    os.makedirs(nanochat_dir, exist_ok=True)
    return nanochat_dir

logger.info(f"ベースディレクトリ: {get_base_dir()}")

## 事前学習データセットの構築

事前学習データセットは、トークナイザーの訓練とモデルの事前学習に使用するデータセット

[karpathy/fineweb-edu-100b-shuffle][2]を使用:

- [FineWeb-EDUデータセット][1]を、シャッフルしシャード（断片ファイル）に再パッケージ
- 各シャードは25万文字のparquetファイルで、約100MB
- 合計シャード数は1822
- depth=20のモデルを訓練するために必要なシャード数は240で、約24GB

[1]: https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu
[2]: https://huggingface.co/datasets/karpathy/fineweb-edu-100b-shuffle

In [ ]:
import os
import argparse
import time
import requests
import pyarrow.parquet as pq
from multiprocessing import Pool

In [ ]:
# データセットのURL
BASE_URL = "https://huggingface.co/datasets/karpathy/fineweb-edu-100b-shuffle/resolve/main"
logger.info(f"データセットURL: {BASE_URL}")

# データセットの最大シャードインデックス
# fineweb-edu-100b-shuffleの場合、1823個のシャードがあり、インデックスは0から1822まで
MAX_SHARD = 1822
logger.info(f"データセットの最大シャードインデックス: {MAX_SHARD}")

# ダウンロードするシャードの数
num_files = 8
logger.info(f"ダウンロードするシャード数: {num_files}")

# ダウンロードに使用するワーカー数（プロセス数）
num_workers = 4
logger.info(f"ダウンロードに使用するワーカー数: {num_workers}")

# データセットの保存ディレクトリの作成
base_dir = get_base_dir()
DATA_DIR = os.path.join(base_dir, "base_data")
os.makedirs(DATA_DIR, exist_ok=True)
logger.info(f"データセット保存ディレクトリ: {DATA_DIR}")

In [ ]:
def index_to_filename(index):
    """
    シャードのインデックスからファイル名を作成

    Args:
        index (int): シャードのインデックス
    Returns:
        str: ファイル名
    """
    return f"shard_{index:05d}.parquet"

index_to_filename(0)

In [ ]:
def download_single_file(index):
    """
    単一のシャードをダウンロードする

    Args:
        index (int): ダウンロードするシャードのインデックス
    Returns:
        bool: ダウンロードが成功したかどうか
    """
    logger.info(f"シャードのダウンロードを開始 {index=}")

    # ダウンロード先のファイルパスを作成
    filename = index_to_filename(index)
    filepath = os.path.join(DATA_DIR, filename)
    logger.debug(f"保存先 {filepath=}")

    # 既にファイルが存在する場合はスキップ
    if os.path.exists(filepath):
        logger.info(f"ダウンロード済みのためスキップ {index=}")
        return True

    # ダウンロードURLを作成
    url = f"{BASE_URL}/{filename}"
    logger.debug(f"ダウンロードURL {url=}")

    # ダウンロード（最大5回のリトライ）
    max_attempts = 5
    for attempt in range(1, max_attempts + 1):
        try:
            # ストリームモードでダウンロード
            logger.info(f"ダウンロード試行 {attempt}/{max_attempts} {index=}")
            response = requests.get(url, stream=True, timeout=30)
            response.raise_for_status()

            temp_path = filepath + f".tmp"
            logger.debug(f"一時ファイルに書き込み {temp_path=}")
            with open(temp_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)

            os.rename(temp_path, filepath)
            logger.info(f"ダウンロード完了 {index=}")
            return True

        except (requests.RequestException, IOError) as e:
            logger.warning(f"ダウンロードエラー {e} {index=} {attempt=}")

            # 不完全なファイルを削除
            for path in [filepath + f".tmp", filepath]:
                if os.path.exists(path):
                    try:
                        os.remove(path)
                    except:
                        pass

            # バックオフ
            # 2^0, 2^1, 2^2, ...秒待機してリトライ
            if attempt < max_attempts:
                wait_time = 2 ** attempt
                logger.info(f"バックオフ: {wait_time} 秒待機中")
                time.sleep(wait_time)
            else:
                logger.info(f"シャードインデックス {index} のダウンロードに失敗")
                return False

    return False

download_single_file(0)

In [ ]:
# 並列ダウンロード

# シャードインデックスのリストを作成
num = MAX_SHARD + 1 if num_files == -1 else min(num_files, MAX_SHARD + 1)
ids_to_download = list(range(num))
logger.info(f"ダウンロードするシャードインデックス: {ids_to_download=}")

# 並列処理でダウンロードを実行
with Pool(processes=num_workers) as pool:
    results = pool.map(download_single_file, ids_to_download)

# 結果を表示
successful = sum(1 for success in results if success)
logger.info(f"並列ダウンロード完了 {successful/len(ids_to_download)=}")

## トークナイザーの実装

RustBPETokenizerは、rustbpeをラップしたトークナイザークラス

In [ ]:
import pickle
import rustbpe
import tiktoken
import os
import copy
from functools import lru_cache

In [ ]:
SPECIAL_TOKENS = [
    # 事前学習で使用
    "<|bos|>", # 文の開始
    # ファインチューニング時に使用
    "<|user_start|>", # ユーザーメッセージ
    "<|user_end|>",
    "<|assistant_start|>", # アシスタントメッセージ
    "<|assistant_end|>",
    "<|python_start|>", # アシスタントがPython REPLツールを呼び出す
    "<|python_end|>",
    "<|output_start|>", # Python REPLがアシスタントに出力を返す
    "<|output_end|>",
]

logger.info(f"特殊トークン: {SPECIAL_TOKENS=}")

In [ ]:
SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
logger.info(f"事前トークン化の正規表現: {SPLIT_PATTERN=}")

In [ ]:
class RustBPETokenizer:
    "訓練時はrustbpeを使い、推論時はtiktokenを使うトークナイザークラス"

    def __init__(self, enc, bos_token):
        logger.info(f"RustBPETokenizer初期化開始 {enc=} {bos_token=}")

        # tiktokenのEncodingオブジェクト
        self.enc = enc 

        # BOSトークンIDをキャッシュ
        self.bos_token_id = self.encode_special(bos_token)
        logger.info(f"RustBPETokenizer初期化完了 {self.bos_token_id=}")

    @classmethod
    def train_from_iterator(cls, text_iterator, vocab_size):
        logger.info(f"RustBPETokenizerの訓練開始 {vocab_size=}")

        # 1) rustbpeを訓練

        # rustbpeのTokenizerオブジェクトを作成
        tokenizer = rustbpe.Tokenizer()

        # 特殊トークンは後で__init__で挿入されるため、ここでは訓練しない
        vocab_size_no_special = vocab_size - len(SPECIAL_TOKENS)

        assert vocab_size_no_special >= 256, f"vocab_size_no_special must be at least 256, got {vocab_size_no_special}"

        # トークナイザーを訓練
        tokenizer.train_from_iterator(text_iterator, vocab_size_no_special, pattern=SPLIT_PATTERN)

        # 2) tiktokenのエンコーダーを構築

        # 事前トークン化の正規表現パターン
        pattern = tokenizer.get_pattern()

        # マージルール
        # {バイト列: マージの優先順位ランク}
        mergeable_ranks_list = tokenizer.get_mergeable_ranks()
        mergeable_ranks = {bytes(k): v for k, v in mergeable_ranks_list}

        # 特殊トークンをマージルールの後に追加
        # {トークン名: トークンID}
        tokens_offset = len(mergeable_ranks)
        special_tokens = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}

        # tiktokenのエンコーダーを構築
        enc = tiktoken.Encoding(
            name="rustbpe",
            pat_str=pattern,
            mergeable_ranks=mergeable_ranks,
            special_tokens=special_tokens,
        )

        logger.info(f"RustBPETokenizerの訓練完了")

        # RustBPETokenizerオブジェクトを返す
        return cls(enc, "<|bos|>")

    @classmethod
    def from_directory(cls, tokenizer_dir):
        logger.info(f"RustBPETokenizerをディレクトリから読み込み開始 {tokenizer_dir=}")

        pickle_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
        logger.debug(f"トークナイザーファイルパス {pickle_path=}")

        with open(pickle_path, "rb") as f:
            enc = pickle.load(f)

        logger.info(f"RustBPETokenizerをディレクトリから読み込み完了 {enc=}")
        return cls(enc, "<|bos|>")

    @classmethod
    def from_pretrained(cls, tiktoken_name):
        logger.info(f"学習済みのtiktokenトークナイザーを読み込み開始 {tiktoken_name=}")

        # https://github.com/openai/tiktoken/blob/eedc8563/tiktoken_ext/openai_public.py
        enc = tiktoken.get_encoding(tiktoken_name)

        # nanochatでは<|bos|>を使用するが、tiktokenのgpt2などでは<|endoftext|>が使用されているため

        logger.info(f"学習済みのtiktokenトークナイザーを読み込み完了 {enc=}")
        return cls(enc, "<|endoftext|>")

    def get_vocab_size(self):
        return self.enc.n_vocab

    def get_special_tokens(self):
        return self.enc.special_tokens_set

    def id_to_token(self, id):
        return self.enc.decode([id])

    @lru_cache(maxsize=32)
    def encode_special(self, text):
        return self.enc.encode_single_token(text)

    def get_bos_token_id(self):
        return self.bos_token_id

    def encode(self, text, prepend=None, append=None, num_threads=8):
        logger.info(f"エンコード開始 {text=} {prepend=} {append=} {num_threads=}")

        # 接頭辞の特殊トークンIDを取得
        if prepend is not None:
            prepend_id = prepend if isinstance(prepend, int) else self.encode_special(prepend)
            logger.debug(f"{prepend_id=}")

        # 接尾辞の特殊トークンIDを取得
        if append is not None:
            append_id = append if isinstance(append, int) else self.encode_special(append)
            logger.debug(f"{append_id=}")

        # 入力が文字列の場合
        if isinstance(text, str):

            # tiktokenでエンコード
            ids = self.enc.encode_ordinary(text)

            # 特殊トークンを追加
            if prepend is not None:
                ids.insert(0, prepend_id) # TODO: slightly inefficient here? :( hmm
            if append is not None:
                ids.append(append_id)

        # 入力が文字列のリストの場合
        elif isinstance(text, list):

            # tiktokenでバッチエンコード
            ids = self.enc.encode_ordinary_batch(text, num_threads=num_threads)

            # 特殊トークンを追加
            if prepend is not None:
                for ids_row in ids:
                    ids_row.insert(0, prepend_id) # TODO: same
            if append is not None:
                for ids_row in ids:
                    ids_row.append(append_id)
        else:
            raise ValueError(f"Invalid input type: {type(text)}")

        logger.info(f"エンコード完了 {ids=}")
        return ids

    def __call__(self, *args, **kwargs):
        return self.encode(*args, **kwargs)

    def decode(self, ids):
        logger.info(f"デコード開始 {ids=}")

        # tiktokenでデコード
        res = self.enc.decode(ids)

        logger.info(f"デコード完了 {res=}")
        return res

    def save(self, tokenizer_dir):
        logger.info(f"トークナイザーの保存開始 {tokenizer_dir=}")

        os.makedirs(tokenizer_dir, exist_ok=True)
        pickle_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
        logger.debug(f"トークナイザーファイルパス {pickle_path=}")

        with open(pickle_path, "wb") as f:
            pickle.dump(self.enc, f)

        logger.info(f"トークナイザーの保存完了 {pickle_path=}")

    def render_conversation(self, conversation, max_tokens=2048):
        """
        チャット形式のデータをトークン化する

        Args:
            conversation: dict チャット会話データ
            max_tokens: int 最大トークン数
        Returns:
            ids: list[int] トークンIDのリスト
            mask: list[int] 同じ長さのマスクリスト、mask=1はアシスタントが学習すべきトークン
        """
        logger.info(f"会話のレンダリング開始 {conversation=} {max_tokens=}")

        # ids, masks that we will return and a helper function to help build them up.
        ids, mask = [], []

        def add_tokens(token_ids, mask_val):
            if isinstance(token_ids, int):
                token_ids = [token_ids]
            ids.extend(token_ids)
            mask.extend([mask_val] * len(token_ids))

        # sometimes the first message is a system message...
        # => just merge it with the second (user) message
        if conversation["messages"][0]["role"] == "system":
            # some conversation surgery is necessary here for now...
            conversation = copy.deepcopy(conversation) # avoid mutating the original
            messages = conversation["messages"]
            assert messages[1]["role"] == "user", "System message must be followed by a user message"
            messages[1]["content"] = messages[0]["content"] + "\n\n" + messages[1]["content"]
            messages = messages[1:]
        else:
            messages = conversation["messages"]

        assert len(messages) >= 1, f"Conversation has less than 1 message: {messages}"

        # fetch all the special tokens we need
        bos = self.get_bos_token_id()
        user_start, user_end = self.encode_special("<|user_start|>"), self.encode_special("<|user_end|>")
        assistant_start, assistant_end = self.encode_special("<|assistant_start|>"), self.encode_special("<|assistant_end|>")
        python_start, python_end = self.encode_special("<|python_start|>"), self.encode_special("<|python_end|>")
        output_start, output_end = self.encode_special("<|output_start|>"), self.encode_special("<|output_end|>")

        # 開始トークンを追加
        add_tokens(bos, 0)

        # メッセージを順に処理
        for i, message in enumerate(messages):

            # some sanity checking here around assumptions, to prevent footguns
            must_be_from = "user" if i % 2 == 0 else "assistant"
            assert message["role"] == must_be_from, f"Message {i} is from {message['role']} but should be from {must_be_from}"

            # content can be either a simple string or a list of parts (e.g. containing tool calls)
            content = message["content"]

            # ユーザーメッセージの場合
            if message["role"] == "user":
                assert isinstance(content, str), "User messages are simply expected to be strings"
                value_ids = self.encode(content)

                # 損失計算に含めない
                add_tokens(user_start, 0)
                add_tokens(value_ids, 0)
                add_tokens(user_end, 0)

            # アシスタントメッセージの場合
            elif message["role"] == "assistant":

                # アシスタントの開始トークンを追加
                add_tokens(assistant_start, 0)

                if isinstance(content, str):
                    # テキストをトークン化
                    value_ids = self.encode(content)

                    # 損失計算に含める
                    add_tokens(value_ids, 1)

                elif isinstance(content, list):
                    for part in content:
                        # テキストをトークン化
                        value_ids = self.encode(part["text"])

                        # 通常のテキストの場合
                        if part["type"] == "text":

                            # 損失計算に含める
                            add_tokens(value_ids, 1)

                        # Pythonのツール呼び出しの場合
                        elif part["type"] == "python":
                            # ツールの開始と終了トークンを追加
                            add_tokens(python_start, 1)
                            add_tokens(value_ids, 1)
                            add_tokens(python_end, 1)

                        # Pythonの出力の場合
                        elif part["type"] == "python_output":
                            # ツールの出力の開始と終了トークンを追加
                            # 推論時はPythonの出力を用いるため損失計算に含めない
                            add_tokens(output_start, 0)
                            add_tokens(value_ids, 0)
                            add_tokens(output_end, 0)
                        else:
                            raise ValueError(f"Unknown part type: {part['type']}")
                else:
                    raise ValueError(f"Unknown content type: {type(content)}")
                add_tokens(assistant_end, 1)

        # 最大トークン数に切り詰める
        ids = ids[:max_tokens]
        mask = mask[:max_tokens]

        logger.info(f"会話のレンダリング完了 {ids=} {mask=}")
        return ids, mask

    def visualize_tokenization(self, ids, mask, with_token_id=False):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'
        tokens = []
        for i, (token_id, mask_val) in enumerate(zip(ids, mask)):
            token_str = self.decode([token_id])
            color = GREEN if mask_val == 1 else RED
            tokens.append(f"{color}{token_str}{RESET}")
            if with_token_id:
                tokens.append(f"{GRAY}({token_id}){RESET}")
        return '|'.join(tokens)

    def render_for_completion(self, conversation):
        """
        Used during Reinforcement Learning. In that setting, we want to
        render the conversation priming the Assistant for a completion.
        Unlike the Chat SFT case, we don't need to return the mask.
        """
        # We have some surgery to do: we need to pop the last message (of the Assistant)
        conversation = copy.deepcopy(conversation) # avoid mutating the original
        messages = conversation["messages"]
        assert messages[-1]["role"] == "assistant", "Last message must be from the Assistant"
        messages.pop() # remove the last message (of the Assistant) inplace

        # Now tokenize the conversation
        ids, mask = self.render_conversation(conversation)

        # Finally, to prime the Assistant for a completion, append the Assistant start token
        assistant_start = self.encode_special("<|assistant_start|>")
        ids.append(assistant_start)
        return ids

## トークナイザーの訓練

In [ ]:
import os
import time
import argparse
import torch

In [ ]:
# 訓練に使用する最大文字数
# デフォルトは10B（100億）文字だが、多いので2B（20億）文字に設定
max_chars = 2_000_000_000
logger.info(f"トークナイザーの訓練に使用する最大文字数: {max_chars=}")

# 1ドキュメントあたりの最大文字数
# デフォルトは10,000文字
doc_cap = 10_000
logger.info(f"1ドキュメントあたりの最大文字数: {doc_cap=}")

# トークナイザーの語彙数
# デフォルトは65,536（2の16乗）
vocab_size = 65_536
logger.info(f"トークナイザーの語彙数: {vocab_size=}")

In [ ]:
def list_parquet_files(data_dir=None):
    """
    データディレクトリ内の全てのparquetファイルのフルパスを返す
    Args:
        data_dir (str, optional): データディレクトリのパス。Noneの場合、デフォルトのDATA_DIRを使用する。
    Returns:
        List[str]: parquetファイルのフルパスのリスト
    """
    # パスを設定
    data_dir = DATA_DIR if data_dir is None else data_dir

    # parquetファイルをリストアップ
    parquet_files = sorted([
        f for f in os.listdir(data_dir)
        if f.endswith('.parquet') and not f.endswith('.tmp')
    ])

    # フルパスを作成
    parquet_paths = [os.path.join(data_dir, f) for f in parquet_files]

    return parquet_paths

list_parquet_files()[:5]

In [ ]:
def parquets_iter_batched(split, start=0, step=1):
    """
    データセットをバッチで読み込むためのイテレータを返す

    splitは、"train"または"val"を指定
    "train"の場合、最初のN-1ファイルを使用
    "val"の場合、最後のファイルを使用
    startとstepは、分散データ処理（Distributed Data Processing）でスキップする際に使用

    Args:
        split (str): "train"または"val"
        start (int, optional): 開始インデックス。デフォルトは0。
        step (int, optional): ステップサイズ。デフォルトは1。
    Yields:
        List[str]: 各バッチのテキストデータのリスト
    """
    logger.info(f"データセットを読み込み開始 {split=} {start=} {step=}")
    assert split in ["train", "val"], "split must be 'train' or 'val'"

    # すべてのparquetファイルのパスを取得
    parquet_paths = list_parquet_files()
    logger.debug(f"全てのparquetファイル {parquet_paths=}")

    # splitに応じてファイルを選択
    parquet_paths = parquet_paths[:-1] if split == "train" else parquet_paths[-1:]
    logger.info(f"使用するparquetファイル: {parquet_paths=}")

    for filepath in parquet_paths:
        # Parquetファイルを読み込む
        pf = pq.ParquetFile(filepath)
        logger.debug(f"{filepath=} {pf.num_row_groups=}")

        # ファイル内の行グループ（row groups）でループ
        # 行グループは、数千から数万行のデータのチャンク
        for rg_idx in range(start, pf.num_row_groups, step):

            # 行グループを読み込む
            rg = pf.read_row_group(rg_idx)
            logger.debug(f"{type(rg)=} {len(rg)=}")

            # テキストカラムをリストとして取得してyield
            texts = rg.column('text').to_pylist()
            logger.debug(f"{type(texts)=} {len(texts)=} {len(texts[0])=}")

            # 1グループ分のテキストを返す
            yield texts

text = parquets_iter_batched("train")
first_row_group = next(text)
first_row_group[0][:50]

In [ ]:
def text_iterator():
    """
    トークナイザーの訓練にテキストデータを1ずつ供給するイテレータ

    1) Flatten the batches into a single iterator
    2) Crop every document to args.doc_cap characters
    3) Break when we've seen args.max_chars characters
    """
    # 文字数カウンタを初期化
    nchars = 0

    # シャードごとにループ
    for batch in parquets_iter_batched(split="train"):

        # シャードのドキュメントごとにループ
        for doc in batch:

            doc_text = doc
            logger.debug(f"ドキュメントからテキストを取得 {len(doc_text)=}")

            # ドキュメントの文字数が10,000文字を超える場合、切り詰める
            if len(doc_text) > doc_cap:
                doc_text = doc_text[:doc_cap]
                logger.debug(f"ドキュメントを切り詰め {len(doc_text)=}")

            # 文字数カウンタを更新
            nchars += len(doc_text)
            logger.debug(f"累積文字数を更新 {nchars=}")

            yield doc_text

            if nchars > max_chars:
                logger.info(f"最大文字数に達したため終了 {nchars=}")
                return

text_iter = text_iterator()
first_doc = next(text_iter)
len(first_doc), first_doc[:50]

In [ ]:
# ログを無効化してトークナイザーを訓練
# （訓練済みの場合は既存のトークナイザーを読み込み）

# トークナイザーの保存ディレクトリを設定
base_dir = get_base_dir()
tokenizer_dir = os.path.join(base_dir, "tokenizer")
logger.info(f"トークナイザーの保存ディレクトリ: {tokenizer_dir}")

if not os.path.exists(tokenizer_dir):
    logger.debug("トークナイザーの訓練を開始")
    logger.setLevel(logging.ERROR)
    t0 = time.time()

    # イテレータを初期化
    text_iter = text_iterator()

    # トークナイザーを訓練
    tokenizer = RustBPETokenizer.train_from_iterator(text_iter, vocab_size)

    t1 = time.time()
    train_time = t1 - t0
    logger.setLevel(logging.DEBUG)
    logger.debug(f"トークナイザーの訓練完了 {train_time=}秒") # 26秒

    # トークナイザーを保存
    tokenizer.save(tokenizer_dir)
else:
    logger.debug("既存のトークナイザーを読み込み")
    tokenizer = RustBPETokenizer.from_directory(tokenizer_dir)

In [ ]:
# トークナイザーを検証

test_text = """Hello world! This is a test.
Numbers: 123, 4567, 89
Contractions: I'm, you're, it's
Special chars: @#$%^&*()
Unicode: こんにちは世界 🌍"""

encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)
assert decoded == test_text

In [ ]:
# BPB（Bits Per Byte）の評価に必要な早見表を作成
# 通常の損失計算は、トークンごとの平均損失だが、BPBはバイトごとの平均損失
# BPB = 総損失/総バイト数
# これにより、異なるトークナイザーを使用した場合でも、モデルの性能を公平に比較できる

vocab_size = tokenizer.get_vocab_size()
logger.info(f"トークナイザーの語彙数: {vocab_size=}")

special_set = set(tokenizer.get_special_tokens())
logger.info(f"特殊トークン数: {len(special_set)=}")

token_bytes_path = os.path.join(tokenizer_dir, "token_bytes.pt")
logger.info(f"早見表の保存パス: {token_bytes_path=}")

if not os.path.exists(token_bytes_path):
    logger.setLevel(logging.ERROR)

    # トークンIDを文字列にデコード
    token_strings = [tokenizer.decode([token_id]) for token_id in range(vocab_size)]
    token_bytes = []

    for token_id in range(vocab_size):
        # 文字列を取得
        token_str = token_strings[token_id]

        # 特殊トークンの場合
        if token_str in special_set:
            # 特殊トークンは0バイトとして早見表に追加
            token_bytes.append(0)
        else:
            # UTF-8でエンコードした際のバイト数を計算し早見表に追加
            id_bytes = len(token_str.encode("utf-8"))
            token_bytes.append(id_bytes)

    # テンソルに変換
    token_bytes = torch.tensor(token_bytes, dtype=torch.int32, device='cpu')

    # 早見表を保存
    with open(token_bytes_path, "wb") as f:
        torch.save(token_bytes, f)

    logger.setLevel(logging.DEBUG)
else:
    token_bytes = torch.load(token_bytes_path)

# 早見表の統計情報を表示
token_bytes_nonzero = (token_bytes[token_bytes > 0]).to(dtype=torch.float32)
logger.info(f"トークンあたりの最小バイト数 {int(token_bytes_nonzero.min().item())}")
logger.info(f"トークンあたりの最大バイト数 {int(token_bytes_nonzero.max().item())}")
logger.info(f"トークンあたりのバイト数の平均 {token_bytes_nonzero.mean().item()}")
logger.info(f"トークンあたりのバイト数の標準偏差 {token_bytes_nonzero.std().item()}")

# 検証
token_id = 5000
token_str = tokenizer.decode([token_id])
num_bytes = token_bytes[token_id].item()
logger.info(f"Token ID: {token_id}, Token: '{token_str}', Bytes: {num_bytes}")

## トークナイザーの評価

In [ ]:
# テスト用ニュース記事テキスト

news_text = r"""
(Washington, D.C., July 9, 2025)- Yesterday, Mexico’s National Service of Agro-Alimentary Health, Safety, and Quality (SENASICA) reported a new case of New World Screwworm (NWS) in Ixhuatlan de Madero, Veracruz in Mexico, which is approximately 160 miles northward of the current sterile fly dispersal grid, on the eastern side of the country and 370 miles south of the U.S./Mexico border. This new northward detection comes approximately two months after northern detections were reported in Oaxaca and Veracruz, less than 700 miles away from the U.S. border, which triggered the closure of our ports to Mexican cattle, bison, and horses on May 11, 2025.

While USDA announced a risk-based phased port re-opening strategy for cattle, bison, and equine from Mexico beginning as early as July 7, 2025, this newly reported NWS case raises significant concern about the previously reported information shared by Mexican officials and severely compromises the outlined port reopening schedule of five ports from July 7-September 15. Therefore, in order to protect American livestock and our nation’s food supply, Secretary Rollins has ordered the closure of livestock trade through southern ports of entry effective immediately.

“The United States has promised to be vigilant — and after detecting this new NWS case, we are pausing the planned port reopening’s to further quarantine and target this deadly pest in Mexico. We must see additional progress combatting NWS in Veracruz and other nearby Mexican states in order to reopen livestock ports along the Southern border,” said U.S. Secretary of Agriculture Brooke L. Rollins. “Thanks to the aggressive monitoring by USDA staff in the U.S. and in Mexico, we have been able to take quick and decisive action to respond to the spread of this deadly pest.”
""".strip()

In [ ]:
# テスト用非英語テキスト

korean_text = r"""
정직한 사실 위에, 공정한 시선을 더하다
Herald Korea Times

헤럴드코리아타임즈는 정치, 경제, 사회, 문화 등 한국 사회 전반의 주요 이슈를 심도 있게 다루는 종합 온라인 신문사입니다.

우리는 단순히 뉴스를 전달하는 것이 아니라, 사실(Fact)에 기반한 양측의 시각을 균형 있게 조명하며, 독자 여러분이 스스로 판단할 수 있는 ‘정보의 균형’을 제공합니다.

한국 언론의 오랜 문제로 지적되어 온 정치적 편향, 이념적 왜곡에서 벗어나
오직 정직함과 공정함을 원칙으로 삼는 언론을 지향합니다.
어느 한쪽의 주장만을 확대하거나 감추지 않고,
**모든 쟁점에 대해 ‘무엇이 쟁점인지’, ‘누가 무엇을 주장하는지’, ‘사실은 무엇인지’**를 명확히 전달하는 데 집중합니다.
""".strip()

In [ ]:
# テスト用のコードスニペット

code_text = r"""
class BasicTokenizer(Tokenizer):

    def __init__(self):
        super().__init__()

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes
        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = get_stats(ids)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = merge(ids, pair, idx)
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")
""".strip()

In [ ]:
# テスト用の数学テキスト

math_text = r"""
\documentclass[12pt]{article}
\usepackage{amsmath,amsthm,amssymb}
\usepackage[margin=1in]{geometry}

\newtheorem{theorem}{Theorem}
\newtheorem*{remark}{Remark}

\begin{document}

\begin{center}
{\Large A Cute Identity: The Sum of Cubes is a Square}
\end{center}

\begin{theorem}
For every integer $n \ge 1$,
\[
\sum_{k=1}^{n} k^{3} \;=\; \left(\frac{n(n+1)}{2}\right)^{2}.
\]
\end{theorem}

\begin{proof}[Proof 1 (Induction)]
Let $S(n) = \sum_{k=1}^{n} k^3$. For $n=1$, $S(1)=1=(1\cdot 2/2)^2$, so the base case holds.

Assume $S(n)=\big(\tfrac{n(n+1)}{2}\big)^2$ for some $n\ge 1$.
Then
\[
S(n+1)
= S(n) + (n+1)^3
= \left(\frac{n(n+1)}{2}\right)^2 + (n+1)^3.
\]
Factor out $(n+1)^2$:
\[
S(n+1)
= (n+1)^2\left( \frac{n^2}{4} + (n+1) \right)
= (n+1)^2\left( \frac{n^2 + 4n + 4}{4} \right)
= (n+1)^2\left( \frac{(n+2)^2}{4} \right).
\]
Thus
\[
S(n+1)=\left(\frac{(n+1)(n+2)}{2}\right)^2,
\]
which matches the claimed formula with $n$ replaced by $n+1$. By induction, the identity holds for all $n\ge 1$.
\end{proof}

\begin{proof}[Proof 2 (Algebraic telescoping)]
Recall the binomial identity
\[
(k+1)^4 - k^4 = 4k^3 + 6k^2 + 4k + 1.
\]
Summing both sides from $k=0$ to $n$ telescopes:
\[
(n+1)^4 - 0^4
= \sum_{k=0}^{n}\big(4k^3 + 6k^2 + 4k + 1\big)
= 4\sum_{k=1}^{n}k^3 + 6\sum_{k=1}^{n}k^2 + 4\sum_{k=1}^{n}k + (n+1).
\]
Using the standard sums
\[
\sum_{k=1}^{n}k = \frac{n(n+1)}{2}
\quad\text{and}\quad
\sum_{k=1}^{n}k^2 = \frac{n(n+1)(2n+1)}{6},
\]
solve for $\sum_{k=1}^{n}k^3$ to get
\[
\sum_{k=1}^{n}k^3 = \left(\frac{n(n+1)}{2}\right)^2.
\]
\end{proof}

\begin{remark}
Geometrically, the identity says: ``adding up $1^3,2^3,\dots,n^3$ builds a perfect square’’—namely the square of the $n$th triangular number. This is why one sometimes calls it the \emph{sum-of-cubes is a square} phenomenon.
\end{remark}

\end{document}
""".strip()

In [ ]:
# テスト用の科学テキスト

science_text = r"""
Photosynthesis is a photochemical energy transduction process in which light-harvesting pigment–protein complexes within the thylakoid membranes of oxygenic phototrophs absorb photons and initiate charge separation at the reaction center, driving the linear electron transport chain from water to NADP⁺ via photosystem II, the cytochrome b₆f complex, and photosystem I, concomitantly generating a trans-thylakoid proton motive force utilized by chloroplastic ATP synthase. The light-dependent reactions produce ATP and NADPH, which fuel the Calvin–Benson–Bassham cycle in the stroma, wherein ribulose-1,5-bisphosphate is carboxylated by ribulose-1,5-bisphosphate carboxylase/oxygenase (RuBisCO) to form 3-phosphoglycerate, subsequently reduced and regenerated through a series of enzymatic steps, enabling net assimilation of CO₂ into triose phosphates and ultimately carbohydrates. This process is tightly regulated by photoprotective mechanisms, redox feedback, and metabolite flux, representing a central biochemical pathway coupling solar energy capture to the biosphere’s primary productivity.
""".strip()

In [ ]:
# コーパスから訓練用と検証用のテキストを取得

train_docs = next(parquets_iter_batched(split="train"))
train_text = "\n".join(train_docs)
val_docs = next(parquets_iter_batched(split="val"))
val_text = "\n".join(val_docs)

In [ ]:
# 全てのテキストデータをリストに集約

all_text = [
    ("news", news_text),
    ("korean", korean_text),
    ("code", code_text),
    ("math", math_text),
    ("science", science_text),
    ("fwe-train", train_text),
    ("fwe-val", val_text),
]

In [ ]:
# 訓練済みのRust

def get_tokenizer():
    """
    RustBPETokenizerオブジェクトを返す

    Returns:
        RustBPETokenizer: トークナイザーオブジェクト
    """
    base_dir = get_base_dir()
    tokenizer_dir = os.path.join(base_dir, "tokenizer")
    # return HuggingFaceTokenizer.from_directory(tokenizer_dir)
    return RustBPETokenizer.from_directory(tokenizer_dir)

In [ ]:
# GPT-2とGPT-4のトークナイザーと比較

logger.setLevel(logging.ERROR)

tokenizer_results = {}
vocab_sizes = {}

for tokenizer_name in ["gpt2", "gpt4", "ours"]:

    # トークナイザーを取得
    if tokenizer_name == "gpt2":
        tokenizer = RustBPETokenizer.from_pretrained("gpt2")
    elif tokenizer_name == "gpt4":
        tokenizer = RustBPETokenizer.from_pretrained("cl100k_base")
    else:
        tokenizer = get_tokenizer()

    vocab_sizes[tokenizer_name] = tokenizer.get_vocab_size()
    tokenizer_results[tokenizer_name] = {}

    for name, text in all_text:
        # エンコードとデコードを行い結果を検証
        encoded = tokenizer.encode(text)
        decoded = tokenizer.decode(encoded)
        assert decoded == text

        # UTF-8でエンコードした際のバイト数を計算
        encoded_bytes = text.encode('utf-8')
        ratio = len(encoded_bytes) / len(encoded)
        tokenizer_results[tokenizer_name][name] = {
            'bytes': len(encoded_bytes),
            'tokens': len(encoded),
            'ratio': ratio
        }

logger.setLevel(logging.DEBUG)

In [ ]:
# 可視化

print(f"\n語彙数:")
print(f"GPT-2: {vocab_sizes['gpt2']}")
print(f"GPT-4: {vocab_sizes['gpt4']}")
print(f"Ours: {vocab_sizes['ours']}")

# ANSI color codes
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

def print_comparison(baseline_name, baseline_results, ours_results, all_text):
    print(f"\n{baseline_name}との比較:")
    print("=" * 95)
    print(f"{'Text Type':<10} {'Bytes':<8} {baseline_name:<15} {'Ours':<15} {'Relative':<12} {'Better':<10}")
    print(f"{'':10} {'':8} {'Tokens':<7} {'Ratio':<7} {'Tokens':<7} {'Ratio':<7} {'Diff %':<12}")
    print("-" * 95)

    for name, text in all_text:
        baseline_data = baseline_results[name]
        ours_data = ours_results[name]

        # Calculate relative difference (positive means ours is better, negative means worse)
        # Using tokens: fewer tokens is better, so we calculate (baseline_tokens - ours_tokens) / baseline_tokens
        relative_diff = ((baseline_data['tokens'] - ours_data['tokens']) / baseline_data['tokens']) * 100

        # Determine which has better compression (higher ratio = better)
        if baseline_data['ratio'] > ours_data['ratio']:
            baseline_color, ours_color = GREEN, RED
            better = baseline_name
            diff_color = RED
        elif ours_data['ratio'] > baseline_data['ratio']:
            baseline_color, ours_color = RED, GREEN
            better = "Ours"
            diff_color = GREEN
        else:
            baseline_color, ours_color = "", ""
            better = "Tie"
            diff_color = ""

        print(f"{name:<10} {baseline_data['bytes']:<8} "
              f"{baseline_color}{baseline_data['tokens']:<7}{RESET} "
              f"{baseline_color}{baseline_data['ratio']:<7.2f}{RESET} "
              f"{ours_color}{ours_data['tokens']:<7}{RESET} "
              f"{ours_color}{ours_data['ratio']:<7.2f}{RESET} "
              f"{diff_color}{relative_diff:+7.1f}%{RESET}     "
              f"{better:<10}")

# Print comparisons
print_comparison("GPT-2", tokenizer_results['gpt2'], tokenizer_results['ours'], all_text)
print_comparison("GPT-4", tokenizer_results['gpt4'], tokenizer_results['ours'], all_text)


## 最適化関数の実装

最適化関数は[Muon（ミューオン, Momentum Orthgonalized by Newton-Schulz）][1]を使用

[1]: https://arxiv.org/abs/2502.16982

In [ ]:
import torch
from torch import Tensor
import torch.distributed as dist

In [ ]:
@torch.compile # PyTorch2.0のJITコンパイラを使用
def zeropower_via_newtonschulz5(G: Tensor, steps: int) -> Tensor:
    """
    ニュートン・シュルツ反復法で勾配の直行化を近似する

    行列Gの直交行列は、Gの特異値分解G = USV^Tに対してUV^Tで計算できる
    特異値分解（SVD）は計算コストが高いため近似手法を用いる
    原点における傾きを最大化するように選択された係数を持つ5次の反復法を採用

    Args: 
        G (Tensor): 直行化する行列、形状は(..., m, n)
        steps (int): 反復回数
    Returns:
        Tensor: 直交化された行列、形状は(..., m, n)
    """

    assert G.ndim >= 2

    # 5次反復法のための係数
    a, b, c = (3.4445, -4.7750,  2.0315)

    # 勾配をbfloat16にダウンキャスト
    X = G.bfloat16()

    # 安定化のため行列を横長にする
    if G.size(-2) > G.size(-1):
        X = X.mT

    # 行列のスペクトルノルム（最大の特異値）を1以下に正規化
    X = X / (X.norm(dim=(-2, -1), keepdim=True) + 1e-7)

    # ニュートン・シュワルツ反復法を適用
    for _ in range(steps):
        # A = X X^T
        A = X @ X.mT

        # B = b(X X^T) + c(X X^T)^2
        B = b * A + c * A @ A

        # X_k+1 = a X_k + (B) X_k 
        X = a * X + B @ X

    # 元の形状に戻す
    if G.size(-2) > G.size(-1):
        X = X.mT

    # 直交行列を返す
    return X

In [ ]:
class Muon(torch.optim.Optimizer):
    """
    Muon最適化関数 https://kellerjordan.github.io/posts/muon/

    内部的に標準的なSGDモーメンタムを実行し、その後に直交化の後処理ステップを実行する
    直行化ステップでは、各2Dパラメータの更新が最も近い直交行列に置き換えられる
    各更新を効率的に直交化するために、ニュートン・シュルツ反復を使用する
    これにより、GPU上でbfloat16で安定して実行できる利点がある

    注意:
    - この最適化関数は、埋め込み層、最終全結合層、0次元・1次元パラメータには使用しない（AdamWなどを使用）
    - 4Dの畳み込みフィルタに使用する場合、最後の3つの次元をフラット化すると機能する

    Args:
        lr (float): 内部SGDで使用される学習率
        momentum (float): 内部SGDで使用されるモーメンタム
        nesterov (bool): 内部SGDでNesterovスタイルのモーメンタムを使用するかどうか（推奨）
        ns_steps (int): ニュートン・シュルツ反復のステップ数
    """

    def __init__(self, params, lr=0.02, momentum=0.95, nesterov=True, ns_steps=5):
        """
        Muon最適化関数の初期化

        Args:
            params (iterable): 最適化するパラメータ
            lr (float, optional): 内部SGDで使用される学習率
            momentum (float, optional): 内部SGDで使用されるモーメンタム
            nesterov (bool, optional): 内部SGDでNesterovスタイルのモーメンタムを使用するかどうか
            ns_steps (int, optional): ニュートン・シュルツ反復のステップ数
        """
        # 最適化関数のデフォルト設定
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)

        params: list[Tensor] = [*params]

        param_groups = []

        # 同じ要素数のパラメータをグループ化
        for size in {p.numel() for p in params}:
            group = dict(params=[p for p in params if p.numel() == size])
            param_groups.append(group)

        # 親クラスの初期化
        super().__init__(param_groups, defaults)

    @torch.no_grad()
    def step(self):
        """
        Muon最適化関数の1ステップの更新を実行
        """

        # 各パラメータグループでループ
        for group in self.param_groups:

            params: list[Tensor] = group["params"]

            # 各パラメータでループ
            for p in params:
                # 勾配を取得
                g = p.grad
                assert g is not None

                # モメンタムバッファを取得または初期化
                state = self.state[p]
                if "momentum_buffer" not in state:
                    state["momentum_buffer"] = torch.zeros_like(g)
                buf: Tensor = state["momentum_buffer"]

                # モメンタムバッファを最新の勾配で更新（指数移動平均）
                buf.lerp_(g, 1 - group["momentum"])

                # ネステロフ・モーメンタムを使用する場合、勾配を調整
                g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf

                # 勾配を直交化
                g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])

                # パラメータを更新
                # -group["lr"]: 学習率を負にして減少方向に更新
                # max(1, p.size(-2) / p.size(-1))**0.5: 行列の形状に基づいてスケーリング
                p.add_(g, alpha=-group["lr"] * max(1, p.size(-2) / p.size(-1))**0.5)


In [ ]:
class DistMuon(torch.optim.Optimizer):
    """
    Muon: SGD-momentum + (optional) Nesterov, then orthogonalize the 2D update via Newton–Schulz,
    finally apply aspect-ratio scaled step. Performs its own distributed synchronization:
      - reduce_scatter(AVG) for gradient averaging
      - all_gather to replicate updated weights

    Notes:
      * Designed for 2D parameters (e.g., linear/conv kernels reshaped to 2D). Do not use for 0D/1D
        params like embeddings or scalars.
      * Momentum buffers are maintained only on the 'owner' rank for each parameter (rank chosen
        by block-cyclic assignment below). If you checkpoint optimizer state on a single rank,
        consolidate states beforehand.

    Args:
        params: iterable of Tensors
        lr: learning rate
        momentum: momentum coefficient in [0,1)
        nesterov: if True, Nesterov-style update (g <- lerp(g, buf, momentum)); else use buf
        ns_steps: number of Newton–Schulz iterations for the orthogonalization
    """
    def __init__(self, params, lr: float = 0.02, momentum: float = 0.95,
                 nesterov: bool = True, ns_steps: int = 5):
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)
        params = list(params)
        assert all(p.ndim == 2 for p in params), "Muon expects 2D parameters only"
        rank = dist.get_rank()
        # Group all parameters by their shape
        shapes = sorted({p.shape for p in params}) # sort to ensure consistent / deterministic ordering
        param_groups = []
        for shape in shapes:
            group_params = [p for p in params if p.shape == shape]
            device, dtype = group_params[0].device, group_params[0].dtype
            assert all(p.device == device for p in group_params)
            assert all(p.dtype == dtype for p in group_params)
            if rank == 0:
                print(f"Muon: Grouping {len(group_params)} params of shape {shape}, device {device}, dtype {dtype}")
            param_groups.append(dict(params=group_params, zero_buffer=torch.zeros_like(group_params[0])))
        super().__init__(param_groups, defaults)

    @torch.no_grad()
    def step(self):
        rank = dist.get_rank()
        world_size = dist.get_world_size()

        # Ensure all grads exist
        assert all(p.grad is not None for group in self.param_groups for p in group["params"]), "All params must have grads"

        # Kick off all the reduce scatter operations to average up the gradients across all ranks
        all_reduce_futures = []
        for group in self.param_groups:
            params = group["params"]
            zero_buffer = group["zero_buffer"]
            # Go through params in groups of world_size.
            for base_i in range(0, len(params), world_size):
                # The compute owner of each param is rank i % world_size
                owner_idx = base_i + rank
                # each rank stacks up its chunk of world_size params into a list
                rs_input = [p.grad for p in params[base_i:base_i + world_size]]
                # pad rs_input with the zero buffer to complete the group
                rs_input.extend([zero_buffer] * (world_size - len(rs_input)))
                # the output buffer gets strided across the group based on the rank
                rs_output = params[owner_idx].grad if owner_idx < len(params) else torch.empty_like(zero_buffer)
                # reduce scatter the gradients within this group of world_size params
                work = dist.reduce_scatter(rs_output, rs_input, op=dist.ReduceOp.AVG, async_op=True).get_future()
                all_reduce_futures.append(work)

        # Now each rank computes the update and gathers
        future_idx = 0
        all_gather_futures = []
        for group in self.param_groups:
            params = group["params"]
            zero_buffer = group["zero_buffer"]
            # Go through params in groups of world_size.
            for base_i in range(0, len(params), world_size):
                # The compute owner of each param is rank i % world_size
                owner_idx = base_i + rank # calculate the index of the param that this rank owns
                # Wait for the reduce scatter to complete
                all_reduce_futures[future_idx].wait() # possibly later we could use wait_any polling instead
                future_idx += 1
                # Owner computes the Muon update, result is in its param
                if owner_idx < len(params):
                    p = params[owner_idx]
                    g = p.grad  # now averaged across ranks
                    state = self.state[p]
                    if "momentum_buffer" not in state:
                        state["momentum_buffer"] = torch.zeros_like(g)
                    buf: Tensor = state["momentum_buffer"]
                    buf.lerp_(g, 1.0 - group["momentum"])
                    g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf
                    g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])
                    scale = (max(1.0, p.size(-2) / p.size(-1)) ** 0.5)
                    p.add_(g, alpha=-group["lr"] * scale)
                # Replicate updated parameters to all ranks
                ag_input = params[owner_idx] if owner_idx < len(params) else zero_buffer
                ag_output = params[base_i:base_i + world_size]
                ag_output.extend([torch.empty_like(zero_buffer) for _ in range(world_size - len(ag_output))]) # pad
                work = dist.all_gather(ag_output, ag_input, async_op=True).get_future()
                all_gather_futures.append(work)

        # Wait for all work to finish
        torch.futures.collect_all(all_gather_futures).wait()


In [ ]:
class DistAdamW(torch.optim.Optimizer):
    """
    Distributed AdamW optimizer.
    In the style of ZeRO-2, i.e. sharded optimizer states and gradient reduction
    """
    def __init__(self, param_groups, lr: float = 1e-3, betas: tuple[float, float] = (0.9, 0.999), eps: float = 1e-8, weight_decay: float = 0.01):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(param_groups, defaults)

    @torch.compile
    @torch.no_grad()
    def step(self):
        rank = dist.get_rank()
        world_size = dist.get_world_size()
        reduce_scatter_futures: list[torch.Future] = []
        all_reduce_futures: list[torch.Future] = []
        grad_slices = []
        for group in self.param_groups:
            params: list[Tensor] = group["params"]
            for base_i in range(len(params)):
                grad = params[base_i].grad
                rank_size = grad.shape[0] // world_size
                grad_slice = torch.empty_like(grad[:rank_size])
                reduce_scatter_futures.append(dist.reduce_scatter_tensor(grad_slice, grad, op=dist.ReduceOp.AVG, async_op=True).get_future())
                grad_slices.append(grad_slice)

        idx = 0
        for group in self.param_groups:
            beta1, beta2 = group['betas']
            eps = group['eps']
            wd = group['weight_decay']
            params = group['params']
            for base in range(len(params)):
                reduce_scatter_futures[idx].wait()
                p = params[base]
                rank_size = p.shape[0] // world_size
                p_slice = p[rank * rank_size:(rank + 1) * rank_size]
                lr = group['lr'] * getattr(p, "lr_mul", 1.0)
                state = self.state[p]
                g_slice = grad_slices[idx]
                # State init
                if not state:
                    state['step'] = torch.tensor(0, dtype=torch.int64, device=p.device)
                    state['exp_avg'] = torch.zeros_like(p_slice)
                    state['exp_avg_sq'] = torch.zeros_like(p_slice)
                exp_avg = state['exp_avg']
                exp_avg_sq = state['exp_avg_sq']
                state['step'] += 1
                t = state['step']
                # weight decay
                if wd != 0:
                    eff_weight_decay = lr * wd * getattr(p, "wd_mul", 1.0)
                    p_slice.mul_(1 - eff_weight_decay)
                # update running averages
                exp_avg.mul_(beta1).add_(g_slice, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(g_slice, g_slice, value=1 - beta2)
                # bias corrections
                bias1 = 1 - beta1 ** t
                bias2 = 1 - beta2 ** t
                # compute step
                denom = exp_avg_sq.sqrt().add_(eps)
                step_size = lr * (torch.sqrt(bias2) / bias1)
                update = exp_avg.div(denom).mul_(step_size)
                p_slice.add_(other=update, alpha=-1.0)
                idx += 1
                all_reduce_futures.append(dist.all_gather_into_tensor(p, p_slice, async_op=True).get_future())
        torch.futures.collect_all(all_reduce_futures).wait()

## モデル実装

In [ ]:
import math
from functools import partial
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F

### ヘルパー関数

In [ ]:
def is_ddp():
    """
    分散データ並列（Distributed Data Parallel, DDP）が有効かどうか

    Returns:
        bool: DDPが有効な場合True、そうでない場合False
    """

    # RANK環境変数はtorchrunなどによって設定されるプロセスの識別子
    return int(os.environ.get('RANK', -1)) != -1

is_ddp()

In [ ]:
def get_dist_info():
    """
    分散データ並列（Distributed Data Parallel, DDP）訓練の環境変数を取得
    """
    if is_ddp():
        assert all(var in os.environ for var in ['RANK', 'LOCAL_RANK', 'WORLD_SIZE'])

        # グローバルなプロセス識別子
        ddp_rank = int(os.environ['RANK'])

        # ローカル（同一ノード内）のプロセス識別子
        ddp_local_rank = int(os.environ['LOCAL_RANK'])

        # ワールドサイズ（全プロセス数）
        ddp_world_size = int(os.environ['WORLD_SIZE'])
        return True, ddp_rank, ddp_local_rank, ddp_world_size
    else:
        return False, 0, 0, 1

get_dist_info()

In [ ]:
def norm(x):
    """
    平方根平均二乗ノルム正規化（Root Mean Square Normalization, RMSNorm）を適用する

    Args:
        x (Tensor): 入力テンソル
    Returns:
        Tensor: 正規化されたテンソル
    """
    logger.info(f"RMSNormを適用 {x.shape=}")

    # ゲインやバイアスを持たないため高速
    res = F.rms_norm(x, (x.size(-1),))

    logger.info(f"RMSNorm適用完了 {res.shape=}")
    return res

In [ ]:
def apply_rotary_emb(x, cos, sin):
    """
    回転位置埋め込み（RoPE, Rotary Positional Embedding）を適用する

    Args:
        x (Tensor): 入力テンソル
        cos (Tensor): コサイン成分、形状は(1, seq_len, 1, head_dim/2)
        sin (Tensor): サイン成分、形状は(1, seq_len, 1, head_dim/2)
    Returns:
        Tensor: RoPEが適用されたテンソル、形状は(batch_size, seq_len, num_heads, head_dim)
    """
    logger.info(f"RoPEを適用 {x.shape=}, {x.dtype=} {cos.shape=}, {cos.dtype=} {sin.shape=} {sin.dtype=}")

    assert x.ndim == 4

    # head_dimを2で分割
    d = x.shape[3] // 2

    # 最後の次元を2つに分割
    x1, x2 = x[..., :d], x[..., d:]

    # 2次元回転行列を適用
    y1 = x1 * cos + x2 * sin
    y2 = x1 * (-sin) + x2 * cos
    out = torch.cat([y1, y2], 3) # 最後の次元で結合

    # sinとcosはfloat32で計算されることが多いため、元のデータ型に戻す
    out = out.to(x.dtype)

    logger.info(f"RoPE適用完了 {out.shape=}")
    return out

### GPTConfig

In [ ]:
@dataclass
class GPTConfig:
    sequence_len: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 6 # number of query heads
    n_kv_head: int = 6 # number of key/value heads (MQA)
    n_embd: int = 768

### CausalSelfAttention

In [ ]:
class CausalSelfAttention(nn.Module):
    """
    GQA（Group Query Attention）を実装した因果セルフアテンションモジュール
    """

    def __init__(self, config, layer_idx):
        logger.info(f"CausalSelfAttentionを初期化 {config.n_head=} {config.n_kv_head=} {config.n_embd=} {layer_idx=}")

        super().__init__()

        self.layer_idx = layer_idx

        self.n_head = config.n_head

        self.n_kv_head = config.n_kv_head

        self.n_embd = config.n_embd

        self.head_dim = self.n_embd // self.n_head
        logger.debug(f"{head_dim=}")

        assert self.n_embd % self.n_head == 0
        assert self.n_kv_head <= self.n_head and self.n_head % self.n_kv_head == 0

        self.c_q = nn.Linear(self.n_embd, self.n_head * self.head_dim, bias=False)

        self.c_k = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)

        self.c_v = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)

        self.c_proj = nn.Linear(self.n_embd, self.n_embd, bias=False)

        logger.info("CausalSelfAttentionの初期化完了")

    def forward(self, x, cos_sin, kv_cache):
        logger.info(f"CausalSelfAttentionの順伝搬を実行 {x.shape=} {x.dtype=} {cos_sin[0].shape=} {cos_sin[0].dtype=} {kv_cache if kv_cache is not None else None=}")

        B, T, C = x.size()

        # 1) クエリ・キー・バリューを計算

        q = self.c_q(x).view(B, T, self.n_head, self.head_dim)

        k = self.c_k(x).view(B, T, self.n_kv_head, self.head_dim)

        v = self.c_v(x).view(B, T, self.n_kv_head, self.head_dim)

        # 2) クエリとキーにRoPEを適用

        cos, sin = cos_sin
        q, k = apply_rotary_emb(q, cos, sin), apply_rotary_emb(k, cos, sin)

        # 3) RMSNormを適用

        q, k = norm(q), norm(k)

        # 4) KVキャッシュを適用

        # (B, T, H, D) -> (B, H, T, D)
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        # KVキャッシュがある場合（推論時）
        if kv_cache is not None:
            # KVキャッシュを更新し、これまでの全てのKVを取得
            k, v = kv_cache.insert_kv(self.layer_idx, k, v)
            logger.debug(f"KVキャッシュを更新 {k.shape=} {v.shape=}")

        # 5) スケールド・ドットプロダクト・アテンションを計算

        # クエリの数
        Tq = q.size(2)
        logger.debug(f"{Tq=}")

        # キー/バリューの数（キャッシュ内も含む）
        Tk = k.size(2)
        logger.debug(f"{Tk=}")

        # Attention: queries attend to keys/values autoregressively. A few cases to handle:

        # GQAを有効にするかどうか
        enable_gqa = self.n_head != self.n_kv_head
        logger.debug(f"GQAを有効化: {enable_gqa=}")

        # 訓練時またはクエリ数とキー/バリュー数が等しい場合
        if kv_cache is None or Tq == Tk:
            logger.debug("通常の因果アテンションを適用")
            y = F.scaled_dot_product_attention(q, k, v, is_causal=True, enable_gqa=enable_gqa)

        # 推論時でクエリが1つだけの場合
        elif Tq == 1:
            logger.debug("マスクを使用しない因果アテンションを適用")
            y = F.scaled_dot_product_attention(q, k, v, is_causal=False, enable_gqa=enable_gqa)

        # 推論時で複数のトークンを一度に処理する場合（prefill）
        else:
            logger.debug("マスクを手動で作成して因果アテンションを適用")

            # アテンションマスクを初期化
            attn_mask = torch.zeros((Tq, Tk), dtype=torch.bool, device=q.device)
            logger.debug(f"{attn_mask.shape=}")

            # キャッシュ済みの長さを取得
            prefix_len = Tk - Tq
            logger.debug(f"{prefix_len=}")

            if prefix_len > 0:
                # マスクの左側（キャッシュ部分）をマスクしない
                attn_mask[:, :prefix_len] = True

            # マスクの右側に下三角行列を設定
            attn_mask[:, prefix_len:] = torch.tril(
                torch.ones((Tq, Tq), dtype=torch.bool, device=q.device)
            )

            y = F.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, enable_gqa=enable_gqa)

        # 6) 出力処理

        # (B, H, T, D) -> (B, T, H, D)
        y = y.transpose(1, 2).contiguous().view(B, T, -1)
        logger.debug(f"アテンション出力を整形 {y.shape=}")

        y = self.c_proj(y)
        logger.debug(f"最終線形変換を適用 {y.shape=}")

        logger.info(f"CausalSelfAttentionの順伝搬完了 {y.shape=}")
        return y

### MLP


In [ ]:
class MLP(nn.Module):
    """
    フィードフォワードネットワーク（FFN）モジュール
    """

    def __init__(self, config):
        logger.info(f"MLPを初期化 {config.n_embd=}")
        super().__init__()

        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=False)

        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=False)

        logger.info("MLPの初期化完了")

    def forward(self, x):
        logger.info(f"MLPの順伝搬を実行 {x.shape=} {x.dtype=}")

        x = self.c_fc(x)
        logger.debug(f"{x.shape=}")

        # Squared ReLU活性化関数を適用
        # GELUやSiLU（Swish）よりも高速でメモリ効率が良い
        x = F.relu(x).square()

        x = self.c_proj(x)
        logger.info(f"MLPの順伝搬完了 {x.shape=}")
        return x

### Block

In [ ]:
class Block(nn.Module):
    """
    Transformerブロックモジュール
    """

    def __init__(self, config, layer_idx):
        logger.info(f"Transformer Blockを初期化 {config.n_embd=} {layer_idx=}")
        super().__init__()
        self.attn = CausalSelfAttention(config, layer_idx)
        self.mlp = MLP(config)
        logger.info("Transformer Blockの初期化完了")

    def forward(self, x, cos_sin, kv_cache):
        logger.info(f"Transformer Blockの順伝搬を実行 {x.shape=} {x.dtype=} {cos_sin[0].shape=} {cos_sin[0].dtype=} {kv_cache if kv_cache is not None else None=}")

        x = x + self.attn(norm(x), cos_sin, kv_cache)

        x = x + self.mlp(norm(x))

        logger.info(f"Transformer Blockの順伝搬完了 {x.shape=}")
        return x

### GPT

In [ ]:
class GPT(nn.Module):
    """
    GPTモデル全体
    """

    def __init__(self, config):
        logger.info(f"GPTモデルを初期化 {config.n_layer=} {config.n_head=} {config.n_embd=} {config.vocab_size=} {config.sequence_len=}")

        super().__init__()

        self.config = config

        # Transformerのエンコーダー部分
        self.transformer = nn.ModuleDict({
            # 単語埋め込み層
            "wte": nn.Embedding(config.vocab_size, config.n_embd),

            # Transformerブロックのリスト
            "h": nn.ModuleList(
                [Block(config, layer_idx) for layer_idx in range(config.n_layer)]
            ),
        })

        # 出力の線形層（Language Model Head）
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # To support meta device initialization, we init the rotary embeddings here, but it's fake
        # As for rotary_seq_len, these rotary embeddings are pretty small/cheap in memory,
        # so let's just over-compute them, but assert fail if we ever reach that amount.
        # In the future we can dynamically grow the cache, for now it's fine.

        self.rotary_seq_len = config.sequence_len * 10 # 10X over-compute should be enough, TODO make nicer?
        logger.debug(f"{self.rotary_seq_len=}")

        head_dim = config.n_embd // config.n_head
        logger.debug(f"{head_dim=}")

        # RoPEのcosとsinを事前計算
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)

        # cosをバッファに登録（state_dictには保存しない）
        self.register_buffer("cos", cos, persistent=False)

        # sinをバッファに登録（state_dictには保存しない）
        self.register_buffer("sin", sin, persistent=False)

        logger.info("GPTモデルの初期化完了")

    def init_weights(self):
        logger.info("GPTモデルの重みを初期化")

        # 全てのモジュールに対して初期化関数を適用
        self.apply(self._init_weights)

        # 出力層の重みをゼロに初期化
        torch.nn.init.zeros_(self.lm_head.weight)

        # 全てのTransformerブロックの出力層の重みをゼロに初期化
        for block in self.transformer.h:
            torch.nn.init.zeros_(block.mlp.c_proj.weight)
            torch.nn.init.zeros_(block.attn.c_proj.weight)

        # RoPEのcosとsinを再計算
        head_dim = self.config.n_embd // self.config.n_head
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)
        self.cos, self.sin = cos, sin

        # 埋め込み層の重みをbfloat16にダウンキャスト
        # メモリ使用量を削減
        if self.transformer.wte.weight.device.type == "cuda":
            self.transformer.wte.to(dtype=torch.bfloat16)

        logger.info("GPTモデルの重みの初期化完了")

    def _init_weights(self, module):
        logger.info(f"モジュールの重みを初期化 {module.__class__.__name__=}")

        # 全結合層の場合
        if isinstance(module, nn.Linear):
            # 論文に基づく方法で重みを初期化
            # https://arxiv.org/pdf/2310.17813
            fan_out = module.weight.size(0)
            fan_in = module.weight.size(1)
            std = 1.0 / math.sqrt(fan_in) * min(1.0, math.sqrt(fan_out / fan_in))
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        # 埋め込み層の場合
        elif isinstance(module, nn.Embedding):
            # 標準正規分布で重みを初期化
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0)

        logger.info("モジュールの重みの初期化完了")

    # TODO: bump base theta more, e.g. 100K is more common more recently
    def _precompute_rotary_embeddings(self, seq_len, head_dim, base=10000, device=None):
        logger.info(f"RoPEの事前計算を実行 {seq_len=} {head_dim=} {base=}")

        # デバイスが指定されていない場合
        if device is None:
            # 自動検出
            device = self.transformer.wte.weight.device

        # stride the channels
        channel_range = torch.arange(0, head_dim, 2, dtype=torch.float32, device=device)

        # 逆周波数を計算
        inv_freq = 1.0 / (base ** (channel_range / head_dim))
        logger.debug(f"{inv_freq.shape=}")

        # stride the time steps
        t = torch.arange(seq_len, dtype=torch.float32, device=device)
        logger.debug(f"{t.shape=}")

        # 周波数を計算
        freqs = torch.outer(t, inv_freq)
        logger.debug(f"{freqs.shape=}")

        # cosとsinを計算
        cos, sin = freqs.cos(), freqs.sin()

        # bfloat16に変換
        cos, sin = cos.bfloat16(), sin.bfloat16()

        # ブロードキャストのために次元を追加
        cos, sin = cos[None, :, None, :], sin[None, :, None, :]

        logger.info(f"RoPEの事前計算完了 {cos.shape=} {sin.shape=}")
        return cos, sin

    def get_device(self):
        return self.transformer.wte.weight.device

    def estimate_flops(self):
        """
        モデルのトークンあたりのFLOPsを推定する
        https://arxiv.org/abs/2204.02311
        """
        logger.info("モデルのFLOPsを推定開始")

        nparams = sum(p.numel() for p in self.parameters())
        logger.debug(f"{nparams=}")

        nparams_embedding = self.transformer.wte.weight.numel()
        logger.debug(f"{nparams_embedding=}")

        l, h, q, t = self.config.n_layer, self.config.n_head, self.config.n_embd // self.config.n_head, self.config.sequence_len
        logger.debug(f"{l=}, {h=}, {q=}, {t=}")

        num_flops_per_token = 6 * (nparams - nparams_embedding) + 12 * l * h * q * t

        logger.info(f"モデルのFLOPsの推定完了 {num_flops_per_token=}")
        return num_flops_per_token

    def setup_optimizers(self, unembedding_lr=0.004, embedding_lr=0.2, matrix_lr=0.02, weight_decay=0.0):
        logger.info(f"最適化関数を設定開始 {unembedding_lr=} {embedding_lr=} {matrix_lr=} {weight_decay=}")

        model_dim = self.config.n_embd

        ddp, rank, local_rank, world_size = get_dist_info()

        # トランスフォーマーの行列パラメータ
        matrix_params = list(self.transformer.h.parameters())
        logger.debug(f"{len(matrix_params)=}")

        # トランスフォーマーの埋め込みパラメータ
        embedding_params = list(self.transformer.wte.parameters())

        # 出力層のパラメータ
        lm_head_params = list(self.lm_head.parameters())

        assert len(list(self.parameters())) == len(matrix_params) + len(embedding_params) + len(lm_head_params)

        # Create the AdamW optimizer for the embedding and lm_head
        # Scale the LR for the AdamW parameters by ∝1/√dmodel (having tuned the LRs for 768 dim model)

        # 埋め込み層と出力層はAdamWを使用

        # モデルの次元に基づいて学習率をスケーリング
        dmodel_lr_scale = (model_dim / 768) ** -0.5
        if rank == 0:
            printug(f"Scaling the LR for the AdamW parameters ∝1/√({model_dim}/768) = {dmodel_lr_scale:.6f}")

        adam_groups = [
            dict(params=lm_head_params, lr=unembedding_lr * dmodel_lr_scale),
            dict(params=embedding_params, lr=embedding_lr * dmodel_lr_scale),
        ]

        adamw_kwargs = dict(betas=(0.8, 0.95), eps=1e-10, weight_decay=weight_decay)

        AdamWFactory = DistAdamW if ddp else partial(torch.optim.AdamW, fused=True)

        adamw_optimizer = AdamWFactory(adam_groups, **adamw_kwargs)

        # 行列パラメータはMuonを使用

        muon_kwargs = dict(lr=matrix_lr, momentum=0.95)

        MuonFactory = DistMuon if ddp else Muon

        muon_optimizer = MuonFactory(matrix_params, **muon_kwargs)


        # 2つの最適化関数を1つのリストにまとめる

        optimizers = [adamw_optimizer, muon_optimizer]

        for opt in optimizers:
            for group in opt.param_groups:
                group["initial_lr"] = group["lr"]

        logger.info("最適化関数の設定完了")
        return optimizers

    def forward(self, idx, targets=None, kv_cache=None, loss_reduction='mean'):
        logger.info(f"GPTモデルの順伝搬を実行 {idx.shape=} {idx.dtype=} {targets.shape if targets is not None else None=} {kv_cache if kv_cache is not None else None=}")

        B, T = idx.size()

        # RoPEの準備

        # Grab the rotary embeddings for the current sequence length (they are of shape (1, seq_len, 1, head_dim))
        assert T <= self.cos.size(1), f"Sequence length grew beyond the rotary embeddings cache: {T} > {self.cos.size(1)}"

        assert idx.device == self.cos.device, f"Rotary embeddings and idx are on different devices: {idx.device} != {self.cos.device}"

        assert self.cos.dtype == torch.bfloat16, "Rotary embeddings must be in bfloat16"

        # KVキャッシュが存在する場合、現在のキャッシュ内の位置にRoPEをオフセットする必要がある
        T0 = 0 if kv_cache is None else kv_cache.get_pos()

        # 事前計算されたRoPEを現在のシーケンス長に切り詰める
        cos_sin = self.cos[:, T0:T0+T], self.sin[:, T0:T0+T]
        logger.debug(f"RoPEの切り出し完了 {cos_sin[0].shape=} {cos_sin[1].shape=}")

        # Transformerの順伝播

        # トークンIDを埋め込みに変換
        x = self.transformer.wte(idx)

        # 最初のRMSNormを適用（Pre-LN）
        x = norm(x)

        # 各Transformerブロックを順番に適用
        for block in self.transformer.h:
            x = block(x, cos_sin, kv_cache)

        # 最後のRMSNormを適用
        x = norm(x)

        # 出力の線形層を適用

        # Logit Softcapping
        # ロジットの値が+/-15を超えないように制限する
        softcap = 15

        # 訓練モードの場合
        if targets is not None:

            # 出力の線形層を適用
            logits = self.lm_head(x)

            # logitsにソフトキャップを適用
            logits = softcap * torch.tanh(logits / softcap)

            # logitsをfloat32にアップキャスト
            logits = logits.float()

            # クロスエントロピー損失を計算
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=-1,
                reduction=loss_reduction
            )

            logger.info(f"GPTモデルの順伝搬完了（訓練モード） {loss=}")
            return loss

        # 推論モードの場合
        else:

            # 出力の線形層を適用
            logits = self.lm_head(x)

            # logitsにソフトキャップを適用
            logits = softcap * torch.tanh(logits / softcap)

            logger.info(f"GPTモデルの順伝搬完了（推論モード） {logits.shape=}")
            return logits

    @torch.inference_mode() # 勾配計算を無効化
    def generate(self, tokens, max_tokens, temperature=1.0, top_k=None, seed=42):
        """
        テキストを生成

        Naive autoregressive streaming inference.
        To make it super simple, let's assume:
        - batch size is 1
        - ids and the yielded tokens are simple Python lists and ints
        """
        logger.info(f"テキスト生成を開始 {tokens=} {max_tokens=} {temperature=} {top_k=} {seed=}")

        assert isinstance(tokens, list)

        device = self.get_device()
        logger.debug(f"{device=}")

        rng = None

        if temperature > 0:
            rng = torch.Generator(device=device)
            rng.manual_seed(seed)

        ids = torch.tensor([tokens], dtype=torch.long, device=device) # add batch dim

        # 最大トークン数まで生成
        for _ in range(max_tokens):

            # 1トークンを生成
            # (B, T, vocab_size)
            logits = self.forward(ids)

            # 直近のトークンのロジットを取得
            # (B, vocab_size)
            logits = logits[:, -1, :]

            # Top-Kサンプリングが有効な場合
            if top_k is not None:
                # 上位K個以外のロジットを無限小に設定
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            # 温度が設定されている場合
            if temperature > 0:
                # 確率分布をなめらかにしランダムに1つ選択
                logits = logits / temperature
                probs = F.softmax(logits, dim=-1)
                next_ids = torch.multinomial(probs, num_samples=1, generator=rng)

            # 温度が0の場合
            else:
                # 最も高い確率のトークンを選択
                next_ids = torch.argmax(logits, dim=-1, keepdim=True)

            # 生成したトークンをシーケンスに追加
            ids = torch.cat((ids, next_ids), dim=1)

            # 生成したトークンをPythonのintに変換
            token = next_ids.item()

            logger.info(f"生成トークン: {token}")
            yield token

## データローダー

In [ ]:
from collections import deque
import torch

In [ ]:
def tokenizing_distributed_data_loader(B, T, split, tokenizer_threads=4, tokenizer_batch_size=128, device="cuda"):
    """
    Parquetファイルからテキストをストリーミングし、トークン化し、訓練バッチを作成するデータローダー

    Args:
        B (int): バッチサイズ
        T (int): シーケンス長
        split (str): データセットの分割 ("train" または "val")
        tokenizer_threads (int): トークナイザーのスレッド数
        tokenizer_batch_size (int): トークナイザーのバッチサイズ
        device (str): デバイス ("cuda" または "cpu")

    Yields:
        Tuple[Tensor, Tensor]: 入力テンソルとターゲットテンソルのタプル
    """
    logger.info(f"分散データローダーを初期化 {B=} {T=} {split=} {tokenizer_threads=} {tokenizer_batch_size=} {device=}")

    assert split in ["train", "val"], "split must be 'train' or 'val'"

    ddp, ddp_rank, ddp_local_rank, ddp_world_size = get_dist_info()

    # 必要なトークン数を計算
    # 最後のターゲットを含めるため1を加算
    needed_tokens = B * T + 1
    logger.debug(f"{needed_tokens=}")

    # トークナイザーを取得
    tokenizer = get_tokenizer()

    # BOSトークンを取得
    bos_token = tokenizer.get_bos_token_id()

    # scratch buffer holds the tokens for one iteration
    token_buffer = deque() # we stream tokens on the right and pop from the left

    # infinite iterator over document batches
    def document_batches():
        while True:
            # batch will iterate in group size of the parquet files, usually e.g. 1024 rows
            for batch in parquets_iter_batched(split=split, start=ddp_rank, step=ddp_world_size):
                # for the tokenizer we might want to go in usually smaller batches, e.g. 128 rows
                for i in range(0, len(batch), tokenizer_batch_size):
                    yield batch[i:i+tokenizer_batch_size]
    batches = document_batches()

    batch_index = 0
    while True:
        # Accumulate enough tokens for one iteration before yielding.
        while len(token_buffer) < needed_tokens:
            doc_batch = next(batches)
            token_lists = tokenizer.encode(doc_batch, prepend=bos_token, num_threads=tokenizer_threads)
            for tokens in token_lists:
                token_buffer.extend(tokens)
            batch_index += 1
        # Move tokens from the deque into the scratch buffer
        tokens = [token_buffer.popleft() for _ in range(needed_tokens)]
        # CUDA supports memory pinning for faster transfers between CPU and GPU:
        scratch = torch.tensor(tokens, dtype=torch.int64, pin_memory=(device == "cuda"))
        # Create the inputs/targets as 1D tensors
        inputs_cpu = scratch[:-1].to(dtype=torch.int32)
        targets_cpu = scratch[1:]
        # Reshape to 2D and move to GPU async
        inputs = inputs_cpu.view(B, T).to(device=device, dtype=torch.int32, non_blocking=True)
        targets = targets_cpu.view(B, T).to(device=device, dtype=torch.int64, non_blocking=True)
        yield inputs, targets

## ベースモデルの訓練

In [ ]:
# Number of processes/GPUs to use
# NPROC_PER_NODE=8
NPROC_PER_NODE=1

In [ ]:
# User settings
run = "dummy" # wandb run name default ("dummy" is special - we won't log to wandb)

# Runtime
device_type = "" # cuda|cpu|mps (empty => autodetect good device type default, in order: CUDA > MPS > CPU)

# Model architecture
depth = 20 # the depth of the Transformer model to train, rest of the kwargs are derived
max_seq_len = 2048 # max context length
# Training horizon. Only one of these 3 will be used, in this order of precedence.
num_iterations = -1 # explicit number of steps of the optimization (-1 = disable)
target_flops = -1.0 # calculate num_iterations to reach target_flops. Useful for scaling laws experiments (-1 = disable)
target_param_data_ratio = 20 # calculate num_iterations to maintain fixed data:param ratio (Chinchilla=20) (-1 = disable)

# Optimization
# device_batch_size = 32 # per-device batch size (set to not OOM)
device_batch_size = 4 # per-device batch size (set to not OOM)

total_batch_size = 524288 # total desired batch size, in #tokens
embedding_lr = 0.2 # learning rate for the embedding parameters (Adam)
unembedding_lr = 0.004 # learning rate for the unembedding parameters (Adam)
weight_decay = 0.0 # weight decay for the embedding/unembedding parameters (Adam)
matrix_lr = 0.02 # learning rate for the matrix parameters (Muon)
grad_clip = 1.0 # gradient clipping value (0.0 = disabled)
warmup_ratio = 0.0 # ratio of iterations for LR warmup
warmdown_ratio = 0.2 # ratio of iterations for LR warmdown
final_lr_frac = 0.0 # final LR is this fraction of the initial LR
# Evaluation
eval_every = 250 # every how many steps to evaluate the model for val bpb
eval_tokens = 20*524288 # number of tokens to evaluate val loss on
core_metric_every = 2000 # every how many steps to evaluate the core metric (-1 = disable)
core_metric_max_per_task = 500 # examples per task in estimating the core metric
sample_every = 2000 # every how many steps to sample from the model
# Output
model_tag = "" # optionally override the model tag for the output checkpoint directory name
# now allow CLI to override the settings via the configurator lol
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
# exec(open(os.path.join('nanochat', 'configurator.py')).read()) # overrides from command line or config file
user_config = {k: globals()[k] for k in config_keys} # will be useful for logging

In [ ]:
def autodetect_device_type():
    # prefer to use CUDA if available, otherwise use MPS, otherwise fallback on CPU
    if torch.cuda.is_available():
        device_type = "cuda"
    elif torch.backends.mps.is_available():
        device_type = "mps"
    else:
        device_type = "cpu"
    print0(f"Autodetected device type: {device_type}")
    return device_type

In [ ]:
def compute_init(device_type="cuda"): # cuda|cpu|mps
    """Basic initialization that we keep doing over and over, so make common."""

    assert device_type in ["cuda", "mps", "cpu"], "Invalid device type atm"
    if device_type == "cuda":
        assert torch.cuda.is_available(), "Your PyTorch installation is not configured for CUDA but device_type is 'cuda'"
    if device_type == "mps":
        assert torch.backends.mps.is_available(), "Your PyTorch installation is not configured for MPS but device_type is 'mps'"

    # Reproducibility
    torch.manual_seed(42)
    if device_type == "cuda":
        torch.cuda.manual_seed(42)
    # skipping full reproducibility for now, possibly investigate slowdown later
    # torch.use_deterministic_algorithms(True)

    # Precision
    if device_type == "cuda":
        torch.set_float32_matmul_precision("high") # uses tf32 instead of fp32 for matmuls

    # Distributed setup: Distributed Data Parallel (DDP), optional, and requires CUDA
    ddp, ddp_rank, ddp_local_rank, ddp_world_size = get_dist_info()
    if ddp and device_type == "cuda":
        device = torch.device("cuda", ddp_local_rank)
        torch.cuda.set_device(device)  # make "cuda" default to this device
        dist.init_process_group(backend="nccl", device_id=device)
        dist.barrier()
    else:
        device = torch.device(device_type) # mps|cpu

    if ddp_rank == 0:
        logger.info(f"Distributed world size: {ddp_world_size}")

    return ddp, ddp_rank, ddp_local_rank, ddp_world_size, device


In [ ]:
def print0(s="",**kwargs):
    ddp_rank = int(os.environ.get('RANK', 0))
    if ddp_rank == 0:
        print(s, **kwargs)

In [ ]:
# Compute init
device_type = autodetect_device_type() if device_type == "" else device_type
ddp, ddp_rank, ddp_local_rank, ddp_world_size, device = compute_init(device_type)
master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) if device_type == "cuda" else nullcontext()
synchronize = torch.cuda.synchronize if device_type == "cuda" else lambda: None
get_max_memory = torch.cuda.max_memory_allocated if device_type == "cuda" else lambda: 0

In [ ]:
# wandb logging init

class DummyWandb:
    """Useful if we wish to not use wandb but have all the same signatures"""
    def __init__(self):
        pass
    def log(self, *args, **kwargs):
        pass
    def finish(self):
        pass

use_dummy_wandb = run == "dummy" or not master_process
wandb_run = DummyWandb() if use_dummy_wandb else wandb.init(project="nanochat", name=run, config=user_config)

In [ ]:
def get_token_bytes(device="cpu"):
    import torch
    base_dir = get_base_dir()
    tokenizer_dir = os.path.join(base_dir, "tokenizer")
    token_bytes_path = os.path.join(tokenizer_dir, "token_bytes.pt")
    assert os.path.exists(token_bytes_path), f"Token bytes not found at {token_bytes_path}? It gets written by tok_train.py"
    with open(token_bytes_path, "rb") as f:
        token_bytes = torch.load(f, map_location=device)
    return token_bytes

In [ ]:
# Tokenizer will be useful for evaluation, also we need the vocab size
tokenizer = get_tokenizer()
token_bytes = get_token_bytes(device=device)
vocab_size = tokenizer.get_vocab_size()
print0(f"Vocab size: {vocab_size:,}")

In [ ]:
# Model kwargs are derived from the desired depth of the model
num_layers = depth
model_dim = depth * 64 # aspect ratio 64 (usually this is varied from 64 -> 128 as model size increases)
num_heads = max(1, (model_dim + 127) // 128) # head dim 128 (the division here is ceil div)
num_kv_heads = num_heads # default is 1:1 GQA (Group Query Attention) ratio (i.e. GQA is disabled)
print0(f"num_layers: {num_layers}")
print0(f"model_dim: {model_dim}")
print0(f"num_heads: {num_heads}")
print0(f"num_kv_heads: {num_kv_heads}")


In [ ]:
# Optimizer / data / training length related hyperparameters
# figure out the needed gradient accumulation to reach the desired total batch size
tokens_per_fwdbwd = device_batch_size * max_seq_len # tokens per iteration for a single rank
world_tokens_per_fwdbwd = tokens_per_fwdbwd * ddp_world_size # total tokens per iteration for all ranks
assert total_batch_size % world_tokens_per_fwdbwd == 0
grad_accum_steps = total_batch_size // world_tokens_per_fwdbwd
print0(f"Tokens / micro-batch / rank: {device_batch_size} x {max_seq_len} = {tokens_per_fwdbwd:,}")
print0(f"Tokens / micro-batch: {world_tokens_per_fwdbwd:,}")
print0(f"Total batch size {total_batch_size:,} => gradient accumulation steps: {grad_accum_steps}")

In [ ]:
# -----------------------------------------------------------------------------
# Initialize the Model
model_config_kwargs = dict(sequence_len=max_seq_len, vocab_size=vocab_size, n_layer=num_layers, n_head=num_heads, n_kv_head=num_kv_heads, n_embd=model_dim)
with torch.device("meta"):
    model_config = GPTConfig(**model_config_kwargs)
    model = GPT(model_config)
model.to_empty(device=device)
model.init_weights()
orig_model = model # original, uncompiled model, for saving raw model state_dict
model = torch.compile(model, dynamic=False) # TODO: dynamic True/False think through
num_params = sum(p.numel() for p in model.parameters())
print0(f"Number of parameters: {num_params:,}")
num_flops_per_token = model.estimate_flops()
print0(f"Estimated FLOPs per token: {num_flops_per_token:e}")


In [ ]:
# Calculate number of iterations. Either it is given, or from target flops, or from target data:param ratio (in that order)
assert num_iterations > 0 or target_param_data_ratio > 0 or target_flops > 0
if num_iterations > 0:
    print0(f"Using user-provided number of iterations: {num_iterations:,}")
elif target_flops > 0:
    # calculate the number of iterations from the target flops
    num_iterations = round(target_flops / (num_flops_per_token * total_batch_size))
    print0(f"Calculated number of iterations from target FLOPs: {num_iterations:,}")
elif target_param_data_ratio > 0:
    # calculate the number of iterations from the target param data ratio
    target_tokens = target_param_data_ratio * num_params
    num_iterations = target_tokens // total_batch_size
    print0(f"Calculated number of iterations from target data:param ratio: {num_iterations:,}")
else:
    raise ValueError("No training horizon specified")
total_tokens = total_batch_size * num_iterations
print0(f"Total number of training tokens: {total_tokens:,}")
print0(f"Tokens : Params ratio: {total_batch_size * num_iterations / num_params:.2f}") # Chinchilla is ~20
print0(f"Total training FLOPs estimate: {num_flops_per_token * total_tokens:e}")

In [ ]:
# -----------------------------------------------------------------------------
# Initialize the Optimizer (Muon for Linear layers, AdamW for embedding and lm_head)
optimizers = model.setup_optimizers(unembedding_lr=unembedding_lr, embedding_lr=embedding_lr, matrix_lr=matrix_lr, weight_decay=weight_decay)
adamw_optimizer, muon_optimizer = optimizers

In [ ]:
# Initialize the DataLoaders for train/val
base_dir = get_base_dir()
tokens_dir = os.path.join(base_dir, "tokenized_data")
train_loader = tokenizing_distributed_data_loader(device_batch_size, max_seq_len, split="train", device=device)
build_val_loader = lambda: tokenizing_distributed_data_loader(device_batch_size, max_seq_len, split="val", device=device)
x, y = next(train_loader) # kick off load of the very first batch of data

In [ ]:
# -----------------------------------------------------------------------------
# Set up hyperparameter schedulers

# Learning rate scheduler
def get_lr_multiplier(it):
    warmup_iters = round(warmup_ratio * num_iterations)
    warmdown_iters = round(warmdown_ratio * num_iterations)
    if it < warmup_iters:
        return (it + 1) / warmup_iters
    elif it <= num_iterations - warmdown_iters:
        return 1.0
    else:
        progress = (num_iterations - it) / warmdown_iters
        return progress * 1.0 + (1 - progress) * final_lr_frac

# Momentum scheduler for Muon optimizer
def get_muon_momentum(it):
    frac = min(it / 300, 1)
    momentum = (1 - frac) * 0.85 + frac * 0.95
    return momentum

In [ ]:
import math
import torch
import torch.distributed as dist

@torch.no_grad()
def evaluate_bpb(model, batches, steps, token_bytes):
    """
    Instead of the naive 'mean loss', this function returns the bits per byte (bpb),
    which is a tokenization vocab size-indepedent metric, meaning you are still comparing
    apples:apples if you change the vocab size. The way this works is that instead of just
    calculating the average loss as usual, you calculate the sum loss, and indepependently
    also the sum bytes (of all the target tokens), and divide. This normalizes the loss by
    the number of bytes that the target tokens represent.

    The added complexity is so that:
    1) All "normal" tokens are normalized by the length of the token in bytes
    2) No special tokens (e.g. <|bos|>) are included in the metric - they are masked out.
    3) No actively masked tokens (using ignore_index of e.g. -1) are included in the metric.

    In addition to evaluate_loss, we need the token_bytes tensor:
    It is a 1D tensor of shape (vocab_size,), indicating the number of bytes for
    each token id, or 0 if the token is to not be counted (e.g. special tokens).
    """
    # record the losses
    total_nats = torch.tensor(0.0, dtype=torch.float32, device=model.get_device())
    total_bytes = torch.tensor(0, dtype=torch.int64, device=model.get_device())
    batch_iter = iter(batches)
    for _ in range(steps):
        x, y = next(batch_iter)
        loss2d = model(x, y, loss_reduction='none') # (B, T)
        loss2d = loss2d.view(-1) # flatten
        y = y.view(-1) # flatten
        if (y.int() < 0).any(): # mps does not currently have kernel for < 0 for int64, only int32
            # slightly more complex code path if some target tokens are ignore_index (e.g. -1)
            # any target token < 0 is to be ignored: do NOT index token_bytes with negatives
            valid = y >= 0
            y_safe = torch.where(valid, y, torch.zeros_like(y))
            # map valid targets to their byte length; ignored targets contribute 0 bytes
            num_bytes2d = torch.where(
                valid,
                token_bytes[y_safe],
                torch.zeros_like(y, dtype=token_bytes.dtype)
            )
            total_nats += (loss2d * (num_bytes2d > 0)).sum()
            total_bytes += num_bytes2d.sum()
        else:
            # fast path: no ignored targets, safe to index directly
            num_bytes2d = token_bytes[y]
            total_nats += (loss2d * (num_bytes2d > 0)).sum()
            total_bytes += num_bytes2d.sum()
    # sum reduce across all ranks
    world_size = dist.get_world_size() if dist.is_initialized() else 1
    if world_size > 1:
        dist.all_reduce(total_nats, op=dist.ReduceOp.SUM)
        dist.all_reduce(total_bytes, op=dist.ReduceOp.SUM)
    # move both to cpu, calculate bpb and return
    total_nats = total_nats.item()
    total_bytes = total_bytes.item()
    if total_bytes == 0:
        return float('inf')
    bpb = total_nats / (math.log(2) * total_bytes)
    return bpb


In [ ]:
# -----------------------------------------------------------------------------
# Training loop
min_val_bpb = float("inf")
smooth_train_loss = 0 # EMA of training loss
ema_beta = 0.9 # EMA decay factor
total_training_time = 0 # total wall-clock time of training
# note that we run +1 steps only so that we can eval and save at the end
# for step in range(num_iterations + 1):
for step in range(3):
    last_step = step == num_iterations
    flops_so_far = num_flops_per_token * total_batch_size * step

    # once in a while: evaluate the val bpb (all ranks participate)
    if last_step or step % eval_every == 0:
        model.eval()
        val_loader = build_val_loader()
        eval_steps = eval_tokens // (device_batch_size * max_seq_len * ddp_world_size)
        with autocast_ctx:
            val_bpb = evaluate_bpb(model, val_loader, eval_steps, token_bytes)
        print0(f"Step {step:05d} | Validation bpb: {val_bpb:.4f}")
        if val_bpb < min_val_bpb:
            min_val_bpb = val_bpb
        wandb_run.log({
            "step": step,
            "total_training_flops": flops_so_far,
            "total_training_time": total_training_time,
            "val/bpb": val_bpb,
        })
        model.train()

    # once in a while: estimate the CORE metric (all ranks participate)
    # use the original uncompiled model because the inputs keep changing shape
    results = {}
    if core_metric_every > 0 and (last_step or (step > 0 and step % core_metric_every == 0)):
        model.eval()
        with autocast_ctx:
            results = evaluate_model(orig_model, tokenizer, device, max_per_task=core_metric_max_per_task)
        print0(f"Step {step:05d} | CORE metric: {results['core_metric']:.4f}")
        wandb_run.log({
            "step": step,
            "total_training_flops": flops_so_far,
            "core_metric": results["core_metric"],
            "centered_results": results["centered_results"],
        })
        model.train()

    # once in a while: sample from the model (only on master process)
    # use the original uncompiled model because the inputs keep changing shape
    if master_process and (last_step or (step > 0 and step % sample_every == 0)):
        model.eval()
        prompts = [
            "The capital of France is",
            "The chemical symbol of gold is",
            "If yesterday was Friday, then tomorrow will be",
            "The opposite of hot is",
            "The planets of the solar system are:",
            "My favorite color is",
            "If 5*x + 3 = 13, then x is",
        ]
        engine = Engine(orig_model, tokenizer) # use orig_model to avoid recompilation
        for prompt in prompts:
            tokens = tokenizer(prompt, prepend="<|bos|>")
            with autocast_ctx:
                sample, _ = engine.generate_batch(tokens, num_samples=1, max_tokens=16, temperature=0)
            print0(tokenizer.decode(sample[0]))
        model.train()

    # save checkpoint at the end of the run (only on master process)
    if master_process and last_step:
        output_dirname = model_tag if model_tag else f"d{depth}" # e.g. d12
        checkpoint_dir = os.path.join(base_dir, "base_checkpoints", output_dirname)
        save_checkpoint(
            checkpoint_dir,
            step,
            orig_model.state_dict(),
            [opt.state_dict() for opt in optimizers], # TODO: make sure saving across ranks is done correctly
            {
                "step": step,
                "val_bpb": val_bpb, # loss at last step
                "model_config": model_config_kwargs,
                "user_config": user_config, # inputs to the training script
                "device_batch_size": device_batch_size,
                "max_seq_len": max_seq_len,
            }
        )

    if last_step:
        break

    # -------------------------------------------------------------------------
    # single training step
    # evaluate the gradient
    synchronize()
    t0 = time.time()
    for micro_step in range(grad_accum_steps):
        with autocast_ctx:
            loss = model(x, y)
        train_loss = loss.detach() # for logging
        loss = loss / grad_accum_steps # each .backward() is a grad sum => normalize loss here
        loss.backward()
        x, y = next(train_loader) # prefetch the next batch while the GPU is busy with forward/backward
    # gradient clipping
    grad_clip_enabled = grad_clip > 0.0
    if grad_clip_enabled:
        grad_norm_tensor = torch.nn.utils.clip_grad_norm_(orig_model.parameters(), grad_clip)
        grad_norm = grad_norm_tensor.item() # GPU tensor -> CPU float (note: cpu-gpu sync point)
    # step the optimizers
    lrm = get_lr_multiplier(step)
    for opt in optimizers:
        for group in opt.param_groups:
            group["lr"] = group["initial_lr"] * lrm
    muon_momentum = get_muon_momentum(step)
    for group in muon_optimizer.param_groups:
        group["momentum"] = muon_momentum
    for opt in optimizers:
        opt.step()
    model.zero_grad(set_to_none=True)
    synchronize()
    t1 = time.time()
    dt = t1 - t0
    # -------------------------------------------------------------------------

    # logging
    smooth_train_loss = ema_beta * smooth_train_loss + (1 - ema_beta) * train_loss.item() # EMA the training loss
    debiased_smooth_loss = smooth_train_loss / (1 - ema_beta**(step + 1)) # debias the EMA
    pct_done = 100 * step / num_iterations
    tok_per_sec = int(total_batch_size / dt)
    flops_per_sec = num_flops_per_token * total_batch_size / dt
    promised_flops_per_sec_h100 = 989e12 * ddp_world_size # bfloat16 H100 SXM and without 2:4 sparsity
    mfu = 100 * flops_per_sec / promised_flops_per_sec_h100 # in %
    if step > 10:
        total_training_time += dt # only count the time after the first 10 steps
    print_grad_norm = f" grad norm: {grad_norm:.4f} |" if grad_clip_enabled else ""
    print0(f"step {step:05d}/{num_iterations:05d} ({pct_done:.2f}%) | loss: {debiased_smooth_loss:.6f} |{print_grad_norm} lrm: {lrm:.2f} | dt: {dt * 1000:.2f}ms | tok/sec: {tok_per_sec:,} | mfu: {mfu:.2f} | total time: {total_training_time/60:.2f}m")
    if step % 100 == 0:
        log_data = {
            "step": step,
            "total_training_flops": flops_so_far,
            "total_training_time": total_training_time,
            "train/loss": debiased_smooth_loss,
            "train/lrm": lrm,
            "train/dt": dt,
            "train/tok_per_sec": tok_per_sec,
            "train/mfu": mfu,
        }
        if grad_clip_enabled:
            log_data["train/grad_norm"] = grad_norm
        wandb_run.log(log_data)